In [1]:
pip install transformers



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U imbalanced-learn


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1rc0
    Uninstalling keras-2.13.1rc0:
      Successfully uninstalled keras-2.13.1rc0

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 8.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0rc0
    Uninstalling tensorflow-estimator-2.13.0rc0:
      Successfully uninstalled tensorflow-estimator-2.13.0rc0
  Attempting uninstall: tensorflow-macos
    Found existing installation: tensorflow-macos 2.13.0rc1
    Uninstalling tensorflow-macos-2.13.0rc1:
      Successfully uninstalled tensorflow-macos-2.13.0rc1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.13.0rc1
    Uninstalling tensorflow-2.13.0rc1:
      Successfully uninstalled tensorflow-2.13.0rc1

[notice] A new rel

In [7]:
pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 2.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
import numpy as np
import nltk
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import ADASYN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.layers import Attention, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import RepeatVector
import seaborn as sns  # Add this import

nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Mount Google Drive to access files
# from google.colab import drive
# drive.mount('/content/drive')

# Load CSV dataset
data = pd.read_csv('Equalised_eclipse_jdt_250each.csv')

# Count the number of samples for each class
class_counts = data['Priority'].value_counts()

# Find the class with the minimum number of samples
minority_class = class_counts.idxmin()

print("Minority class:", minority_class)
print(class_counts)


# Text preprocessing steps
def preprocess_text(text):
    if isinstance(text, str):
        # Lowercase conversion
        text = text.lower()

        # Remove punctuation and special characters
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = text.split()
        tokens = [word for word in tokens if word not in stop_words]
        text = ' '.join(tokens)

        # Stemming
        stemmer = PorterStemmer()
        tokens = text.split()
        tokens = [stemmer.stem(word) for word in tokens]
        text = ' '.join(tokens)

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = text.split()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        text = ' '.join(tokens)

    return text

# Apply text preprocessing to 'description' and 'title' columns
data['Description'] = data['Description'].apply(preprocess_text)
data['Title'] = data['Title'].apply(preprocess_text)

# Tokenize using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_sequence_length = 3000

def tokenize_text(text):
    if isinstance(text, (str, int, float)):
        text = str(text)  # Convert text to string if it's not already

        tokens = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_sequence_length, padding='max_length', truncation=True)
        input_ids = tokens['input_ids']
        attention_mask = tokens['attention_mask']
    else:
        input_ids = []
        attention_mask = []
    return input_ids, attention_mask

# Apply tokenization to 'Description' and 'Title' columns
data['Description'] = data['Description'].apply(tokenize_text)
data['Title'] = data['Title'].apply(tokenize_text)

# Convert tokenized data to numpy arrays
input_ids = np.array([item[0] for item in data['Description']])
attention_masks = np.array([item[1] for item in data['Description']])

# Reshape input IDs and attention masks to have at most 2 dimensions
input_ids = input_ids.reshape(input_ids.shape[0], -1)
attention_masks = attention_masks.reshape(attention_masks.shape[0], -1)

# Concatenate input IDs and attention masks
X_input = np.concatenate((input_ids, attention_masks), axis=1)
y_input = data['Priority']

# # Apply ADASYN oversampling
# adasyn = ADASYN()
# X_resampled, y_resampled = adasyn.fit_resample(X_input, y_input)

# # Apply ADASYN oversampling
# if len(class_counts) > 1 and class_counts.iloc[-1] < class_counts.iloc[0]:
#     adasyn = ADASYN(n_neighbors=5, sampling_strategy='minority')
#     X_resampled, y_resampled = adasyn.fit_resample(X_input, y_input)
# else:
#     X_resampled, y_resampled = X_input, y_input

X_resampled, y_resampled = X_input, y_input

# Separate the resampled input IDs and attention masks
input_ids_resampled = X_resampled[:, :max_sequence_length]
attention_masks_resampled = X_resampled[:, max_sequence_length:]

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(input_ids_resampled, y_resampled, test_size=0.2, random_state=42)

# Pad sequences
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

from sklearn.preprocessing import LabelEncoder

# Encode string labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to categorical
num_classes = len(label_encoder.classes_)
y_train = to_categorical(y_train_encoded, num_classes)
y_test = to_categorical(y_test_encoded, num_classes)

# Build HAN model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(len(tokenizer.get_vocab()) + 1, 128)(input_layer)
word_encoder = Bidirectional(GRU(64, return_sequences=True))(embedding_layer)
word_attention = Attention()([word_encoder, word_encoder])
word_flatten = Flatten()(word_attention)
word_dense = Dense(64)(word_flatten)
word_repeat = RepeatVector(max_sequence_length)(word_dense)
sentence_encoder = Bidirectional(GRU(64, return_sequences=True))(word_repeat)
sentence_attention = Attention()([sentence_encoder, sentence_encoder])
sentence_flatten = Flatten()(sentence_attention)
sentence_dense = Dense(64)(sentence_flatten)
output_layer = Dense(num_classes, activation='softmax')(sentence_dense)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10, callbacks=[early_stopping])

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()

# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])
plt.show()

# Calculate precision, recall, and F1-score for each class
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_original = label_encoder.inverse_transform(y_test_encoded)
y_pred_original = label_encoder.inverse_transform(y_pred_classes)

precision = precision_score(y_test_original, y_pred_original, average=None, labels=label_encoder.classes_)
recall = recall_score(y_test_original, y_pred_original, average=None, labels=label_encoder.classes_)
f1 = f1_score(y_test_original, y_pred_original, average=None, labels=label_encoder.classes_)

# Display precision, recall, and F1-score for each class
for i, class_name in enumerate(label_encoder.classes_):
    print(f"Class: {class_name}")
    print(f"Precision: {precision[i]:.4f}")
    print(f"Recall: {recall[i]:.4f}")
    print(f"F1-score: {f1[i]:.4f}")
    print("="*30)

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test_original, y_pred_original, labels=label_encoder.classes_)
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Print model summary
model.summary()

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


Minority class: P1
P4    263
P3    256
P2    252
P5    252
P1    251
Name: Priority, dtype: int64
Epoch 1/10


2023-08-13 10:45:58.604875: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-08-13 10:45:58.605527: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 5

 6/32 [====>.........................] - ETA: 8:05 - loss: 699.5507 - accuracy: 0.1823

KeyboardInterrupt: 